In [1]:
%run ./'01 load.ipynb'

Last data for germany: 2020-04-04 18:52:06


In [2]:
country = 'Germany'

In [3]:
df = df[df['country'] == country]

In [4]:
df['growth'] = df['confirmed'].diff().fillna(0)

In [5]:
curr = df.tail(1)['date'].dt.strftime('%d. %b, %H:%M Uhr').values[0]

In [6]:
# Newest data
df.tail(10)

,date,date_string,country,confirmed,active,deaths,recovered,growth
4874,2020-03-26 23:59:59,26. Mar,Germany,"43,938",0,267,0,"6,615"
4875,2020-03-27 23:59:59,27. Mar,Germany,"50,871",0,342,0,"6,933"
4876,2020-03-28 23:59:59,28. Mar,Germany,"57,695",0,433,0,"6,824"
4877,2020-03-29 23:59:59,29. Mar,Germany,"62,095",0,533,0,"4,400"
4878,2020-03-30 23:59:59,30. Mar,Germany,"66,885",0,645,0,"4,790"
4879,2020-03-31 23:59:59,31. Mar,Germany,"71,808",0,775,0,"4,923"
4880,2020-04-01 23:59:59,01. Apr,Germany,"77,872",0,920,0,"6,064"
4881,2020-04-02 23:59:59,02. Apr,Germany,"84,794",0,"1,107",0,"6,922"
4882,2020-04-03 23:59:59,03. Apr,Germany,"91,159",0,"1,275",0,"6,365"
4883,2020-04-04 18:52:06,04. Apr,Germany,"92,150","64,420","1,330","26,400",991


In [7]:
# Load predictions from 18th march
prediction = pd.read_csv('20200318 Predict Corona - Sheet1.csv')
prediction['predict'] = prediction['predict'].str.replace(',','').astype('str')
prediction['date'] = prediction['date'] + ' 23:59:59'
prediction['date'] = pd.to_datetime(prediction['date'])
prediction['date_string'] = prediction['date'].dt.strftime('%d. %b')

In [8]:
df = pd.merge(left=df, right=prediction[['date', 'predict']], on='date', how='left')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(1,1,1)
ax1.grid(True)
ax1.set_yscale('log')
ax1.plot(df['date_string'][:-1], df['confirmed'][:-1], color = 'blue', linewidth=4)
ax1.plot(df['date_string'][-2:], df['confirmed'][-2:], color = 'blue', linewidth=1)
ax1.plot(df['date_string'], df['predict'], color = 'red', linewidth=2)
ax2 = ax1.twinx()
ax2.plot(df['date_string'][:-1], df['confirmed'][:-1], color = 'green', linewidth=4)
ax2.plot(df['date_string'][-2:], df['confirmed'][-2:], color = 'green', linewidth=1)
ax2.plot(df['date_string'], df['predict'], color = 'red', linewidth=2)
ax2.plot(df['date_string'][:-1], df['growth'][:-1], color = 'orange', linewidth=4)
ax2.plot(df['date_string'][-2:], df['growth'][-2:], color = 'orange', linewidth=1)
fig.legend([
    'Bestätigte Fälle (log)',
    'Bestätigte Fälle (log) / ' + curr,
    'Vorhersage vom 18.03. (log)',
    'Bestätigte Fälle (abs)',
    'Bestätigte Fälle (abs) / ' + curr,
    'Vorhersage vom 18.03. (abs)',
    'Zuwachs',
    'Zuwachs / ' + curr], loc='upper left')
ax1.set_xticklabels(labels=df['date_string'], rotation=45)
labels = ax1.xaxis.get_ticklabels()
for label in labels:
    label.set_visible(False)
for label in labels[::7]:
    label.set_visible(True)
fig.tight_layout()
fig.show()